Practicing the xgboost model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import xgboost as xgb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
train.head()

In [ ]:
train.target.value_counts()

In [ ]:
train.shape

In [ ]:
labels = train.target.values

In [ ]:
#train.drop("target",axis=1,inplace=True)

In [ ]:
train.shape

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

In [ ]:
test.head()

In [ ]:
test_id = test.id.values

In [ ]:
train.drop("id",inplace=True,axis=1)
test.drop("id",inplace=True,axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
def modelfit(alg,dtrain,predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgb_param["num_class"] = 9
        xgtrain = xgb.DMatrix(dtrain[predictors].values,label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param,xgtrain,num_boost_round=alg.get_params()["n_estimators"],nfold=cv_folds,metrics="auc",early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    alg.fit(dtrain[predictors].values,dtrain[target].values,eval_metric="auc")
    dtrain_predictions = alg.predict(dtrain[predictors].values)
    dtrain_probabilities = alg.predict_proba(dtrain[predictors].values)[:,1]
    print ("Model Report\n")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target].values, dtrain_probabilities))
    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
target = 'target'
IDcol = 'id'

In [ ]:
predictors = [x for x in train.columns if x not in [target, IDcol]]

In [ ]:
xgb1 = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 500,
    max_depth = 5,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective= 'multi:softmax',
    nthread=4,
    scale_pos_weight=1,
    seed=27
)

In [ ]:
train.head()

In [ ]:
l = train.target.value_counts().index.tolist()

In [ ]:
d = dict(zip(l,range(10)))

In [ ]:
d

In [ ]:
train.target = train.target.replace(d)

In [ ]:
train.target.value_counts()

In [ ]:
train.shape

In [ ]:
modelfit(xgb1,train,predictors)